In [1]:
import os
import sys

In [3]:
pwd

'c:\\Users\\arpit\\OneDrive\\Desktop\\Chicken-Diseases-Classification-with-Fecal-Images\\research'

In [4]:
os.chdir("../")

In [18]:
pwd

'c:\\Users\\arpit\\OneDrive\\Desktop\\Chicken-Diseases-Classification-with-Fecal-Images'

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("allandclive/chicken-disease-1")

print("Path to dataset files:", path)

c:\Users\arpit\anaconda3\envs\tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 266M/266M [00:43<00:00, 6.39MB/s] 

Extracting files...


Path to dataset files: C:\Users\arpit\.cache\kagglehub\datasets\allandclive\chicken-disease-1\versions\1


In [13]:
import shutil
import os

# Define source and destination paths
source_path = r"C:\Users\arpit\.cache\kagglehub\datasets\allandclive\chicken-disease-1\versions\1"
destination_path = r"C:\Users\arpit\OneDrive\Desktop\Chicken-Diseases-Classification-with-Fecal-Images\data"

# Ensure the destination directory exists
os.makedirs(destination_path, exist_ok=True)

# Copy all files and folders from source to destination
shutil.move(source_path, destination_path)

print(f"Dataset moved successfully to {destination_path}")

Dataset moved successfully to C:\Users\arpit\OneDrive\Desktop\Chicken-Diseases-Classification-with-Fecal-Images\data


In [ ]:
dataset_link = "https://github.com/ArpitKadam/Chicken-Diseases-Classification-with-Fecal-Images/raw/refs/heads/main/Chicken-fecal-images.zip"

In [25]:
import tensorflow as tf

# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")

# Check if TensorFlow is built with CUDA
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Number of GPUs Available: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        details = tf.config.experimental.get_device_details(gpu)
        gpu_name = details.get("device_name", "Unknown GPU")
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("No GPU available!")

# Check if TensorFlow can run on GPU
print("\nTensorFlow GPU Execution Test:")
print("GPU Available for TensorFlow:", tf.test.is_gpu_available())

# Check which device TensorFlow operations will run on
print("Default Device:", tf.test.gpu_device_name())

TensorFlow Version: 2.10.1
Built with CUDA: True
Number of GPUs Available: 1
GPU 1: NVIDIA GeForce RTX 3050 Laptop GPU

TensorFlow GPU Execution Test:
GPU Available for TensorFlow: True
Default Device: /device:GPU:0


# DATA INGESTION STEP

### Entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Configuration Manager

In [15]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

### Components

In [16]:
import os
import urllib.request as request
import zipfile
from src.logger.logger import logger
from src.utils.common import get_size
from pathlib import Path


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info("Attempt to download data")
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        logger.info(f"Unzipping file from {self.config.local_data_file} into {unzip_path}")
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Unzipping completed")
        logger.info(f"File successfully extracted to {unzip_path}")

### Pipeline

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-19 16:02:02,375: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-19 16:02:02,379: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-19 16:02:02,381: INFO: common: created directory at: artifacts]
[2025-03-19 16:02:02,384: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-19 16:02:02,386: INFO: 2904967687: Attempt to download data]
[2025-03-19 16:02:12,500: INFO: 2904967687: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9EC7:1CC0FF:26623:3F484:67DA9D20
Accept-Ranges: bytes
Date: Wed, 19 Mar 2025 10

# BASE MODEL PREPARATION STEP

### Entity

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_epochs: int
    params_batch_size: int
    params_batch_size: int

### Configuration Manager

In [32]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES,
            params_epochs=self.params.EPOCHS
        )

        return prepare_base_model_config

### Components

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        logger.info(f"Base model saved at {path}")

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_all is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

### Pipeline

In [34]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-03-19 18:26:51,856: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-19 18:26:51,860: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-19 18:26:51,863: INFO: common: created directory at: artifacts]
[2025-03-19 18:26:51,865: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 10s 0us/step
[2025-03-19 18:27:03,808: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-03-19 18:27:03,908: INFO: 3283328933: Base model saved at artifacts\prepare_base_model\base_model.h5]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block

# MODEL TRAINING STEP